In [8]:
# Setup
!pip install --quiet --upgrade pip

# Install the dependencies.
!pip install --quiet -r requirements.txt

# Restart the runtime by ending the process.
exit()

In [3]:
from __future__ import annotations
import os

import ee
import google.auth

import folium
from serving import data
import json
from branca.element import Figure
from folium import plugins
import sys
from importlib import reload
from serving.data import get_input_image_ee, get_varied_labels
from serving.constants import  BUCKET, PROJECT, CROP, SCALE, IMAGE_BATCH
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.cloud import storage
import logging
import randomname

In [4]:
current_dir = os.getcwd()
sys.path.append(current_dir+'/serving')
logging.basicConfig(filename=f"image_processing.log", level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [5]:
# Google cloud parameters
project = PROJECT
bucket = BUCKET
location = "us-west2-a"  # @param {type:"string"}

# Quick input validations.
assert project, "⚠️ Please provide a Google Cloud project ID"
assert bucket, "⚠️ Please provide a Cloud Storage bucket name"
assert not bucket.startswith(
    "gs://"
), f"⚠️ Please remove the gs:// prefix from the bucket name: {bucket}"
assert location, "⚠️ Please provide a Google Cloud location"

# Set GOOGLE_CLOUD_PROJECT for google.auth.default().
os.environ["GOOGLE_CLOUD_PROJECT"] = project

# Set the gcloud project for other gcloud commands.
!gcloud config set project {project}

# Initialise goofle earth engine
credentials, _ = google.auth.default()
ee.Initialize(
    credentials.with_quota_project(None),
    project=project,
    opt_url="https://earthengine-highvolume.googleapis.com",
)

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [4]:
#CONSTANTS
YEAR_START=2017
YEAR_END=2023

SEASON_START=5
SEASON_END=10

CROP=1 #Corn

# True vision params
vis_params = {
    "min": 0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"],}

vis_agro = {
     "min": 225,
    "max": 4000,
    "bands": ['B11', 'B8', 'B3']}

In [5]:
reload(data)
from serving.data import get_input_image_ee, get_varied_labels

In [64]:
# Show sample filter

start_year = 2016
start_month = 5
end_month = 8
crop_type = 1 # Corn
county = "Jeff Davis".capitalize()
state_fips = "13"
county_fips = "161"

county_geom = (
    ee.FeatureCollection("TIGER/2018/Counties")
    .filter(ee.Filter.eq("COUNTYFP", "161"))
    .filter(ee.Filter.eq("STATEFP", state_fips))
).geometry()

coords = county_geom.centroid().coordinates().getInfo()

s2_img_start = get_input_image_ee(county, county_fips, state_fips, crop_type, start_year, start_month)["image"]
s2_img_end = get_input_image_ee(county, county_fips, state_fips, crop_type, start_year, end_month)["image"]


In [65]:
# Create the map
m1 = folium.Map(coords[::-1])

# Add the CDL layer
layer_left = folium.TileLayer(
    tiles=s2_img_start.getMapId(vis_agro)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name='Cropland Data Layer',
    overlay=True,
    control=True
)

layer_right = folium.TileLayer(
    tiles=s2_img_end.getMapId(vis_agro)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name='Cropland Data Layer',
    overlay=True,
    control=True,
)

sbs = plugins.SideBySideLayers(layer_left=layer_left, layer_right=layer_right)

layer_left.add_to(m1)
layer_right.add_to(m1)
sbs.add_to(m1)    

# Display the map
m1

In [7]:
"""
Changes in crops can be observable in the satellite images. Once can hope that the colour saturation and temporal change would hold predictive power
"""

'\nChanges in crops can be observable in the satellite images. Once can hope that the colour saturation and temporal change would hold predictive power\n'

In [ ]:
# DATA RETRIVAL

In [4]:
def check_blob_prefix_exists(bucket_name, prefix):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    
    blobs = bucket.list_blobs(prefix=prefix, max_results=1)
    return any(blobs)

def batch_check_blobs(bucket_name, prefixes):
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_prefix = {executor.submit(check_blob_prefix_exists, bucket_name, prefix): prefix for prefix in prefixes}
        results = {}
        for future in as_completed(future_to_prefix):
            prefix = future_to_prefix[future]
            results[prefix] = future.result()
    return results

# Generate all prefixes
bucket_name = bucket
labels_df = get_varied_labels(*IMAGE_BATCH)
labels = list(zip(labels_df["county_name"],
                  labels_df["county_ansi"],
             labels_df["state_ansi"],
             labels_df["year"]))
months = [5,7,9]

prefixes = [f'images/{SCALE}/{county.capitalize()}_{fips}/{year}/{month}-{month+1}' 
            for county,_, fips, year in labels
            for month in months]

# Batch check all prefixes
results = batch_check_blobs(bucket_name, prefixes)

# Generate get_input_img_params based on results
get_input_img_params = [
    {"county": county.capitalize(), "county_fips": county_fips, "state_fips": fips, "crop": CROP, "year": year, "month": month}
            for county, county_fips, fips, year in labels
            for month in months
            if not results[f'images/{SCALE}/{county.capitalize()}_{fips}/{year}/{month}-{month+1}']
]

print(f"Number of items to process: {len(get_input_img_params)}")

Number of items to process: 3570


In [5]:
get_input_img_params_small = get_input_img_params[0:1000]
get_input_img_params_small[:2]

[{'county': 'Schuylkill',
  'county_fips': '107',
  'state_fips': '42',
  'crop': 1,
  'year': 2016,
  'month': 5},
 {'county': 'Schuylkill',
  'county_fips': '107',
  'state_fips': '42',
  'crop': 1,
  'year': 2016,
  'month': 7}]

In [6]:
job_name = randomname.get_name(adj=('weather',), noun=('food'))

def export_img(image:ee.Image, image_name: str, county_geom: ee.geometry):
    
    image_name = f"{SCALE}/" + image_name
    img_task = ee.batch.Export.image.toCloudStorage(
    image=image,
    description=image_name.replace("/","-"),
    bucket=bucket,
    fileNamePrefix=f"images/{image_name}",
    scale=SCALE,
    region=county_geom,
    fileDimensions = 4*2048,
    skipEmptyTiles = True,
    fileFormat = "GeoTIFF",
    maxPixels = 1e9
    )

    img_task.start()
    logging.info(f"{job_name}: Export task started for: {image_name}")

def apply_get_input_image(params):
    
    return get_input_image_ee(**params)

def unpack_for_export(list_input):
    county, state_fips = list_input["image_name"].split("/")[0].split("_")
    county_geom = (
    ee.FeatureCollection("TIGER/2018/Counties")
        .filter(ee.Filter.eq("NAME", county.capitalize()))
        .filter(ee.Filter.eq("STATEFP", state_fips))
        ).geometry()
    return export_img(list_input["image"], list_input["image_name"], county_geom)

In [ ]:
logging.info(f"Job {job_name} started")
with beam.Pipeline() as pipeline:
    (
        pipeline
        |"Create parameter sets" >>  beam.Create(get_input_img_params)
        | "Sample counties" >> beam.Map(apply_get_input_image)
        | "Export image to GCS bucket" >> beam.Map(unpack_for_export)
    )
logging.info(f"Job {job_name} finished")

EEException: Too many tasks already in the queue (3000, limit 3000). [while running '[7]: Export image to GCS bucket']